Title
========================================================

In [ ]:
%%R
library(data.table)

In [ ]:
%%R
?data.table

In [ ]:
%%R
df  <- data.frame(x = c("b","b","b","a","a"),v=rnorm(5))
df

In [ ]:
%%R
dt  <- data.table(x = c("b","b","b","a","a"),v=rnorm(5))
dt 

In [ ]:
%%R
CARS  <- data.table(cars)
head(CARS)

In [ ]:
%%R
tables()

In [ ]:
%%R
dt

In [ ]:
%%R
dt[2,]

In [ ]:
%%R
dt[dt$x == "b",]

In [ ]:
%%R
dt["b",]

In [ ]:
%%R
setkey(dt,x)

In [ ]:
%%R
dt

In [ ]:
%%R
haskey(dt)

In [ ]:
%%R
key(dt)

In [ ]:
%%R
dt["b",]

In [ ]:
%%R
dt["b",mult="first"]

In [ ]:
%%R
dt["b",mult="last"]

In [ ]:
%%R
dt["b"]

In [ ]:
%%R
grpsize  <- ceiling(1e7/26^2)
grpsize

In [ ]:
%%R
tt = system.time(df  <- data.frame(
  x = rep(LETTERS, each=26*grpsize),
  y = rep(letters, each=grpsize),
  v = runif(grpsize*26^2),
  stringsAsFactors=F
  ))
tt

In [ ]:
%%R
head(df,3)

In [ ]:
%%R
tail(df,3)

In [ ]:
%%R
dim(df)

Extract arbitrary group from df

In [ ]:
%%R
tt = system.time(
  ans1  <- df[df$x == "R" & df$y == "h",]
  )
tt

In [ ]:
%%R
head(ans1,3)

In [ ]:
%%R
dim(ans1)

Convert to data.table

In [ ]:
%%R
dt  <- as.data.table(df)
system.time(setkey(dt,x,y))

Binary search

In [ ]:
%%R
ss = system.time(
  ans2  <- dt[J("R","h")]
  )
ss
#tt

vector scan

In [ ]:
%%R
system.time(
  ans1  <- dt[x == "R" & y == "h",]  
  )

In [ ]:
%%R
system.time(
  ans2  <- df[df$x == "R" & df$y == "h",]  
  )

In [ ]:
%%R
identical(
  dt[J("R","h"),],
  dt[data.table("R","h"),]
  )

Fast grouping

In [ ]:
%%R
head(dt)
dt[,sum(v)]

When we supply a j expression and a ’by’ list of expressions, the j expression is repeated for each ’by’ group:

In [ ]:
%%R
dt[,sum(v),by=x]

The by in data.table is fast. Let’s compare it to tapply.

In [ ]:
%%R
ttt=system.time(tt <- tapply(dt$v,dt$x,sum)); ttt

In [ ]:
%%R
sss=system.time(ss <- dt[,sum(v),by=x]); sss

In [ ]:
%%R
head(tt)

In [ ]:
%%R
head(ss)

In [ ]:
%%R
as.vector(tt)

In [ ]:
%%R
ss$V1

In [ ]:
%%R
identical(as.vector(tt), ss$V1)

Next, let’s group by two columns:

In [ ]:
%%R
ttt=system.time(tt <- tapply(dt$v,list(dt$x,dt$y),sum)); ttt

In [ ]:
%%R
sss=system.time(ss <- dt[,sum(v),by="x,y"]); sss

In [ ]:
%%R
tt[1:5,1:5]

In [ ]:
%%R
head(ss)

In [ ]:
%%R
identical(
  as.vector(t(tt)),
  ss$V1
  )

t(tt)


==========================================================
http://stackoverflow.com/questions/13493124/outer-join-data-table-r

In [ ]:
%%R
a <- data.table(a=c(1,2,3),b=c(3,4,5)); a
b <- data.table(a=c(1,2),k=c(1,2)); b
setkey(a,a)
setkey(b,a)

key(a)
key(b)

In [ ]:
%%R
b[a,]
a[b,]

TODO:http://stackoverflow.com/questions/12773822/why-does-xy-join-of-data-tables-not-allow-a-full-outer-join-or-a-left-join







================================================

In [ ]:
%%R
require(data.table)

In [ ]:
%%R
# create a regular data.frame
theDF <- data.frame(A=1:10,
                     B=letters[1:10],
                     C=LETTERS[11:20],
                     D=rep(c("One", "Two", "Three"), length.out=10))
theDF

In [ ]:
%%R
# create a data.table
theDT <- data.table(A=1:10,
                     B=letters[1:10],
                     C=LETTERS[11:20],
                     D=rep(c("One", "Two", "Three"), length.out=10))
 # print them and compare
 theDF

Multiple columns in a data.frame should be specified as a character vector. With data.tables the columns should be specified as a list of the actual names, not as characters.

In [ ]:
%%R
theDT[, list(A, C)] 

In [ ]:
%%R
# just one column
theDT[, B]

one column while maintaining data.table structure

In [ ]:
%%R
theDT[, list(B)]

If we must specify the column names as characters (perhaps because they were passed as arguments to a function), the with argument should be set to FALSE.

In [ ]:
%%R
theDT[, "B", with = FALSE]

In [ ]:
%%R
theDT[, c("A", "C"), with = FALSE]

This time we used a vector to hold the column names instead of a list. These nuances are important to proper functions of data.tables but can lead to a great deal of frustration.

##Keys

show tables

In [ ]:
%%R
tables()

The key is used to index the data.table and will provide the extra speed.

set the key

In [ ]:
%%R
setkey(theDT, D)

show the data.table again

In [ ]:
%%R
theDT

The data have been reordered according to column D, which is sorted alphabetically.

In [ ]:
%%R
key(theDT)

In [ ]:
%%R
tables()

This adds some new functionality to selecting rows from data.tables. In addition to selecting rows by the row number or by some expression that evaluates to TRUE or FALSE, a value of the key column can be specified.

In [ ]:
%%R
theDT["One", ]

In [ ]:
%%R
theDT[c("One", "Two"), ]

More than one column can be set as the key.

In [ ]:
%%R
#install.packages("ggplot2")
require(ggplot2)
data(diamonds)
diamondsDT <- data.table(diamonds)
setkey(diamondsDT, cut, color)

In [ ]:
%%R
head(diamondsDT)

To access rows according to both keys, there is a special function named J. It takes multiple arguments, each of which is a vector of values to select.

In [ ]:
%%R
diamondsDT[J("Ideal", "E"), ]

In [ ]:
%%R
diamondsDT[J("Ideal", c("E", "D")), ]

##data.table Aggregation

The primary benefit of indexing is faster aggregation. While aggregate and the various d*ply functions will work because data.tables are just enhanced data.frames, they will be slower than using the built-in aggregation functionality of data.table.



we calculate the mean price of diamonds for each type of cut.

In [ ]:
%%R
aggregate(price ~ cut,  diamonds, mean)

To get the same result using data.table, we do this:

In [ ]:
%%R
diamondsDT[, mean(price), by = cut]

The only difference between this and the previous result is that the columns have different names. To specify the name of the resulting column, pass the aggregation function as a named list.

In [ ]:
%%R
diamondsDT[, list(price = mean(price)), by = cut]

To aggregate on multiple columns, specify them as a list().

In [ ]:
%%R
diamondsDT[, mean(price), by = list(cut, color)]

To aggregate multiple arguments, pass them as a list. Unlike with aggregate, a different metric can be measured for each column.

In [ ]:
%%R
diamondsDT[, list(price = mean(price), carat = mean(carat)), by = cut]

In [ ]:
%%R
diamondsDT[, list(price = mean(price), carat = mean(carat), caratSum = sum(carat)), by = cut]

Finally, both multiple metrics can be calculated and multiple grouping variables can be specified at the same time.

In [ ]:
%%R
diamondsDT[, list(price = mean(price), carat = mean(carat)), by = list(cut, color)]